In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Ziwen 


In [ ]:
# Packages for data processing
import re
import pandas as pd
import string
import spacy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#!pip install wordcloud
from wordcloud import WordCloud

#Package for feature extraction
import nltk.data
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize   
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer



#Packages for building a classifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from statistics import mode

In [ ]:
pip install language-tool-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')

INFO:language_tool_python.download_lt:Unzipping /tmp/tmpzwtf9_ei.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


In [ ]:
!pip install tweet-preprocessor
import preprocessor as p

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def tweet_cleaner(tw_list):
        remove_rt = lambda x: re.sub('RT @\w+: '," ",str(x))
        rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str(x))
        hash = lambda x: re.sub(r'#', "", str(x))
        amp = lambda x: re.sub(r'&amp', "", str(x))

        print("tweet_cleaner")
        print(tw_list[:10])
        tw_list['grammartext'] = tw_list['text'].map(remove_rt).map(rt)
        print("grammar")
        tw_list['clean_text'] = tw_list.text.map(remove_rt).map(rt)
        print("clean_text")
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.HASHTAG)
        tw_list["grammartext"] = tw_list.grammartext.map(p.clean)
        print("grammar")

        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.NUMBER)
        tw_list["clean_text"] = tw_list.clean_text.map(p.clean).map(hash).map(amp)
        tw_list["clean_text"] = tw_list.clean_text.str.lower()
        print('done')
        # print(tw_list)
        return tw_list

In [ ]:
from textblob import TextBlob

In [ ]:
hashtags = ['#VoteThemOut',
            # '#ToryScumOut',
            'F1',
            'Lisa',
            '#EnoughIsEnough',
            '#EnergyPrices',
            '#iOS16',
            '#taiwan',
            '#onepiece',
            '#CostOfLivingCrisis',
            '#GetBackToWorkYouFatPonce', 
            # '#ClosingCeremony',
            '#BookLoversDay',
            '#biden']
# hashtags = ['F1']
for hash in hashtags:
        grammarlist = []
        subjectivitylist = []
        # device = torch.device("cuda")
        df=pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/4/{hash}_USER_scores_with_gender_FINAL_new.csv')
        user_feeds = tweet_cleaner(df)
        tweets = user_feeds.clean_text.to_list()
        print(len(tweets))
        for tweet in tweets:
            matches = int(len(tool.check(str(tweet))))
            grammarlist.append(matches)
            analysis = TextBlob(tweet)
            subjectivitylist.append(analysis.sentiment.subjectivity)
        df['tweet_grammar'] = grammarlist
        df['tweet_subjectivity'] = subjectivitylist

        save_path = f'/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/5/{hash}_USER_scores_with_subjectivity_grammar.csv'

        df.to_csv(save_path)
        

        



tweet_cleaner
   Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1                   id  \
0           0             0               0  1556587785007632391   
1           1             1               1  1556587794582999040   
2           2             2               2  1556587807791087616   
3           3             3               3  1556587818729742338   
4           4             4               4  1556587824710819840   
5           5             5               5  1556587841467064322   
6           6             6               6  1556587942688194560   
7           7             7               7  1556587960929222658   
8           8             8               8  1556587970693562370   
9           9             9               9  1556587978067218433   

                                                text  \
0  RT @sophiee_ob: Customs Nike Airforce 1 Low ’J...   
1  Peter Obi or Peter Obi #VoteThemOut #PeterObiF...   
2  RT @ObidientsYouth: At the mention of OBidient...   
3  RT @Obidie

## Run subjectivity